# 文本生成实战

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=10, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.18.1
pandas 0.25.3
sklearn 0.22.1
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

solve_cudnn_error()

1 Physical GPUs, 1 Logical GPUs


## 数据读取

In [3]:
input_filepath = './data/shakespeare.txt'
text = open(input_filepath, 'r').read()

print(len(text))
print(text[0:100])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


### 数据处理
1. 生成词表 generator vocab
2. 建立字符到id的映射 build mapping char->id
3. 将数据转化为id data->id_data
4. 预测下一个字符的模型，abcd -> bcd\<eos>

In [4]:
# 1.generator vocab
vocab = sorted(set(text)) # 把字符串存为set，则只剩下唯一的字符
print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
# 2.build mapping char->id
char2idx = {char : idx for idx, char in enumerate(vocab)}
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [6]:
idx2char = np.array(vocab)
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [7]:
# 3.data->id_data
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[0:10])
print(text[0:10])

[18 47 56 57 58  1 15 47 58 47]
First Citi


In [8]:
# 4.定义输入输出
def split_input_target(id_text):
    """abcde -> abcd, bcde"""
    return id_text[0:-1], id_text[1:]

# 定义dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
seq_dataset = char_dataset.batch(seq_length + 1, drop_remainder = True)

# test
for ch_id in char_dataset.take(2):
    print(ch_id, idx2char[ch_id.numpy()])
    
for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(repr(''.join(idx2char[seq_id.numpy()])))

tf.Tensor(18, shape=(), dtype=int32) F
tf.Tensor(47, shape=(), dtype=int32) i
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int32)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int32)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


In [9]:
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(2):
    print(item_input.numpy())
    print(item_output.numpy())

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1]
[56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1 58
 53  1 42

In [10]:
# 给dataset做变换
batch_size = 64
buffer_size = 10000

seq_dataset = seq_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

## 模型构建

In [11]:
vocab_size = len(vocab) # 词表大小
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]),
        # 每一步都要返回全部序列，以便和输出的序列做loss计算
        keras.layers.SimpleRNN(units=rnn_units, return_sequences=True),
        keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
simple_rnn (SimpleRNN)       (64, None, 1024)          1311744   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


### 采样测试

In [12]:
# 没经过训练，已经可以进行预测，看一下model的输出
for input_example_batch, target_example_batch in seq_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 65)


In [14]:
# 随机采样 random sampling 或者贪心采样 greedy 获得字符，从而生成一段话
sample_indices = tf.random.categorical(logits=example_batch_predictions[0], num_samples=1)
print(sample_indices) # 采样后的维度 (100, 65) -> (100, 1)

# 将维度展平
sample_indices = tf.squeeze(sample_indices, axis = -1)
print(sample_indices)

tf.Tensor(
[[40]
 [61]
 [ 0]
 [47]
 [29]
 [33]
 [32]
 [22]
 [48]
 [42]
 [ 1]
 [54]
 [10]
 [ 6]
 [25]
 [38]
 [43]
 [23]
 [43]
 [ 5]
 [59]
 [ 3]
 [ 0]
 [ 4]
 [ 0]
 [45]
 [55]
 [50]
 [37]
 [61]
 [13]
 [ 7]
 [23]
 [ 2]
 [21]
 [55]
 [20]
 [16]
 [42]
 [15]
 [64]
 [ 8]
 [60]
 [ 3]
 [56]
 [42]
 [36]
 [11]
 [29]
 [55]
 [43]
 [29]
 [ 5]
 [64]
 [ 7]
 [30]
 [56]
 [16]
 [18]
 [15]
 [64]
 [55]
 [37]
 [57]
 [12]
 [29]
 [15]
 [28]
 [62]
 [ 9]
 [46]
 [24]
 [64]
 [54]
 [33]
 [47]
 [18]
 [ 6]
 [37]
 [21]
 [12]
 [47]
 [29]
 [12]
 [44]
 [63]
 [51]
 [ 6]
 [ 1]
 [21]
 [41]
 [28]
 [49]
 [23]
 [33]
 [15]
 [57]
 [59]
 [11]
 [50]], shape=(100, 1), dtype=int64)
tf.Tensor(
[40 61  0 47 29 33 32 22 48 42  1 54 10  6 25 38 43 23 43  5 59  3  0  4
  0 45 55 50 37 61 13  7 23  2 21 55 20 16 42 15 64  8 60  3 56 42 36 11
 29 55 43 29  5 64  7 30 56 16 18 15 64 55 37 57 12 29 15 28 62  9 46 24
 64 54 33 47 18  6 37 21 12 47 29 12 44 63 51  6  1 21 41 28 49 23 33 15
 57 59 11 50], shape=(100,), dtype=int64)


In [16]:
# 打印输入、输出、预测
print('Input:', repr(''.join(idx2char[input_example_batch[0]])))
print()
print('Output:', repr(''.join(idx2char[target_example_batch[0]])))
print()
print('Predictions:', repr(''.join(idx2char[sample_indices])))

Input: "ment.\nYou thus employ'd, I will go root away\nThe noisome weeds, which without profit suck\nThe soil's"

Output: "ent.\nYou thus employ'd, I will go root away\nThe noisome weeds, which without profit suck\nThe soil's "

Predictions: "bw\niQUTJjd p:,MZeKe'u$\n&\ngqlYwA-K!IqHDdCz.v$rdX;QqeQ'z-RrDFCzqYs?QCPx3hLzpUiF,YI?iQ?fym, IcPkKUCsu;l"


### 训练

In [17]:
# 自定义损失函数，loss从logits里来
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer = 'adam', loss = loss)

example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())

(64, 100)
4.1961346


In [18]:
# 定义文件保存模型
output_dir = 'text_generation_checkpoint'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')

# 定义参数
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix, save_weights_only = True)
epochs = 100
callbacks = [
    checkpoint_callback,
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)
]

# 训练
history = model.fit(seq_dataset, epochs = epochs, callbacks = [checkpoint_callback])

Epoch 1/100
172/172 [==============================] - 39s 225ms/step - loss: 2.7038
Epoch 2/100
172/172 [==============================] - 37s 217ms/step - loss: 2.0595
Epoch 3/100
172/172 [==============================] - 37s 216ms/step - loss: 1.8420
Epoch 4/100
172/172 [==============================] - 37s 216ms/step - loss: 1.7025
Epoch 5/100
172/172 [==============================] - 37s 216ms/step - loss: 1.6059
Epoch 6/100
172/172 [==============================] - 37s 217ms/step - loss: 1.5397
Epoch 7/100
172/172 [==============================] - 38s 219ms/step - loss: 1.4898
Epoch 8/100
172/172 [==============================] - 37s 216ms/step - loss: 1.4510
Epoch 9/100
172/172 [==============================] - 37s 217ms/step - loss: 1.4209
Epoch 10/100
172/172 [==============================] - 37s 218ms/step - loss: 1.3952
Epoch 11/100
172/172 [==============================] - 38s 219ms/step - loss: 1.3709
Epoch 12/100
172/172 [==============================] - 37s 216

In [19]:
tf.train.latest_checkpoint(output_dir)

'text_generation_checkpoint\\ckpt_100'

## 载入模型权重，进行文本生成
文本生成的流程：
- start ch sequence A
- A -> model -> b
- A.append(b) -> B
- B(Ab) -> model -> c
- B.append(c) -> C
- C(Abc) -> model -> ...一直循环结束

In [21]:
model2 = build_model(
    vocab_size,
    embedding_dim,
    rnn_units,
    batch_size = 1)
# 加载权重
model2.load_weights(tf.train.latest_checkpoint(output_dir))
# 定义输入大小
model2.build(tf.TensorShape([1, None]))

model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (1, None, 1024)           1311744   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


In [23]:
# 默认生成1000个字符的文本
def generate_text(model, start_string, num_generate = 1000):
    # 字符变id
    input_eval = [char2idx[ch] for ch in start_string]
    # 扩展一个维度，一维变二维
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    model.reset_states()
    
    # 生成流程：
    # 1. model inference -> predictions
    # 2. sample -> ch -> text_generated.
    # 3. update input_eval
    for _ in range(num_generate):
        # 模型预测 predictions : [batch_size, input_eval_len, vocab_size]
        predictions = model(input_eval)
        # 将第0维消掉
        predictions = tf.squeeze(predictions, 0)
        # 随机采样，predictions_ids: [input_eval_len, 1]
        # 取出预测值中的最后一个字符 a b c -> b c d ,所以取出d
        predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1, 0].numpy()
        text_generated.append(idx2char[predicted_id])
        # 将input_eval替换成predictions，并扩维
        input_eval = tf.expand_dims([predicted_id], 0)
        
    return start_string + ''.join(text_generated)

In [24]:
new_text = generate_text(model2, 'All:')
print(new_text)

All:

E: at.
Thold ondiny thesld:
'dscoorar tot,'

TRK:
TINUDo; metthy g K:
ERLIORAN ig Her'shay aurou hing be wisu tho k!



Whave shw k that me w alinghackitowhinamys halll it, s lerositusterther prtionthr,
A thieade mannthepe sou beng,'llw ame ws windok.

T:
M:
MNTI' wif meng d d bed.
Hered My hal ghen io avind goul lld.
Cithed ORE:

USAMesst, w w bure IE:
I boulyorirouconoume in, whis fy vessthelibes siou f ar.

OMy cucl s Wes t n:
Thesherel anson,
CLI:
CHeraige?
ICibe f INondg, y, veis.
Ofuthadsthan moven:
ELecin,
TERUK:
He RUCow akirde.
Br
ANULee se
AMI athal gou s cosu the

Cikir divead t helio t, anthotie ceny br?
Pldomy w l f O:
A:
fe llf hiratha'dse wind bath beng the


Sheave'ded wno Whe ld s, bes cerd ire wie d ate; e couthame fonsth;
SAUERWhestheny lend J
Thangredas marsthienso y an Wing, atoulertss llighigrdloux:
wind blo Inits ht. if, g, pr TIOLer-whinou INNThe ETERCAt, I brs thng omy artely A: med vee hathenire:

Anonee ig f IICLI
ALO: bll she omowand, p thou,

PE:
Ny L